# Part 3: Single-View Geometry

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [1]:
%matplotlib tk
import matplotlib.pyplot
import numpy
import PIL.Image
import scipy

from sympy import Matrix, solve, symbols

# Provided functions

In [2]:
def get_input_lines(mat: numpy.array, min_lines: int=3) -> tuple[numpy.ndarray, numpy.ndarray, numpy.ndarray]:
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = numpy.zeros((3, 0))
    centers = numpy.zeros((3, 0))

    matplotlib.pyplot.figure()
    matplotlib.pyplot.imshow(mat)

    print("[INFO]: Set at least %d lines to compute vanishing point" % min_lines)
    while True:
        print("[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input")
        clicked = matplotlib.pyplot.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print("[INFO]: Need at least %d lines, you have %d now" % (min_lines, n))
                continue
            else:
                # stop getting lines if number of lines is enough
                break

        # unpack user inputs and save as homogeneous coordinates
        pt1 = numpy.array(object=[clicked[0][0], clicked[0][1], 1])
        pt2 = numpy.array(object=[clicked[1][0], clicked[1][1], 1])
        # get line equation using cross product
        # line equation: line[0] * x + line[1] * y + line[2] = 0
        line = numpy.cross(a=pt1, b=pt2)
        lines = numpy.append(arr=lines, values=line.reshape((3, 1)), axis=1)
        # get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = numpy.append(arr=centers, values=center.reshape((3, 1)), axis=1)

        # plot line segment
        matplotlib.pyplot.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color="cyan", linestyle='-', linewidth=1)

        n += 1

    return n, lines, centers

In [3]:
def plt_lines_vp(mat: numpy.ndarray, lines: numpy.ndarray, vp: numpy.ndarray) -> None:
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(mat.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(mat.shape[0], vp[1] / vp[2]) + 10

    matplotlib.pyplot.rc(group="font", family="serif")
    matplotlib.pyplot.rc(group="text", usetex=True)
    matplotlib.pyplot.figure()
    matplotlib.pyplot.imshow(X=mat)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = numpy.cross(a=numpy.array([1, 0, -bx1]), b=lines[:, i])
            pt2 = numpy.cross(a=numpy.array([1, 0, -bx2]), b=lines[:, i])
        else:
            pt1 = numpy.cross(a=numpy.array([0, 1, -by1]), b=lines[:, i])
            pt2 = numpy.cross(a=numpy.array([0, 1, -by2]), b=lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        matplotlib.pyplot.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color="cyan")

    matplotlib.pyplot.plot(vp[0] / vp[2], vp[1] / vp[2], color="orangered", marker='o')
    matplotlib.pyplot.title(r"Vanishing Point Coordinate: $%f, %f$" % (vp[0], vp[1]))
    matplotlib.pyplot.show()

    return

In [4]:
def get_top_btm_coords(mat: numpy.ndarray, obj: str) -> numpy.ndarray:
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
               where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is
               the homogeneous coordinate of the bottom
    """
    matplotlib.pyplot.figure()
    matplotlib.pyplot.imshow(mat)

    print("[INFO]: Click on the top coordinate of %s" % obj)
    clicked = matplotlib.pyplot.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # uncomment this line "to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print("[INFO]: Click on the bottom coordinate of %s" % obj)
    clicked = matplotlib.pyplot.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    matplotlib.pyplot.plot([x1, x2], [y1, y2], color="yellow")

    return numpy.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [5]:
def get_vanishing_pt(lines: numpy.ndarray) -> numpy.ndarray:
    """
    Solves for the vanishing point using the user-input lines.
    Input:
        lines: np.ndarray of shape (3, n) where each column denotes the parameters of the line equation
               Line equation: line[0] * x + line[1] * y + line[2] = 0
    Return:
        eigvec: vanishing point
    """
    gram_mat = lines@lines.T
    eigvals, eigvecs = scipy.linalg.eig(a=gram_mat)
    idx = numpy.argmin(a=eigvals)
    eigvec = eigvecs[:, idx]
    eigvec /= eigvec[-1]

    return eigvec

In [6]:
def get_horizon_line(vpts: numpy.ndarray) -> numpy.ndarray:
    """
    Calculates the ground horizon line.
    Input:
        vpts: vanishing points
    Return:
        line: np.ndarray of shape (3, 1) where each column denotes the parameters of the line equation
              Line equation: line[0] * x + line[1] * y + line[2] = 0
    """
    pt_0 = vpts[:, 0]
    pt_1 = vpts[:, 1]
    line = numpy.cross(a=pt_0, b=pt_1)
    norm = scipy.linalg.norm(a=line[:2])
    line = line/norm

    return line

In [7]:
def plt_horizon_line(mat: numpy.ndarray, line: numpy.ndarray) -> None:
    """
    Plots the horizon line.
    Inputs:
        mat: image matrix
        line: np.ndarray of shape (3, 1) where each column denotes the parameters of the line equation
              Line equat"on: line[0] * x + line[1] * y + line[2] = 0
    Return:
        None
    """
    w = mat.shape[1]
    x = numpy.arange(w)

    y = (-line[0]*x-line[2])/line[1]

    matplotlib.pyplot.rc(group="font", family="serif")
    matplotlib.pyplot.rc(group="text", usetex=True)
    matplotlib.pyplot.figure()
    matplotlib.pyplot.imshow(X=mat)
    matplotlib.pyplot.plot(x, y, color="cyan", linestyle='-', linewidth=2)
    matplotlib.pyplot.title(r"Horizon Line: $%fx+%fy+%f=0$" % (line[0], line[1], line[2]))
    matplotlib.pyplot.show()

    return

In [8]:
def get_camera_params(vpts: numpy.ndarray) -> tuple[float, float, float]:
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    Input:
        vpts: vanishing points
    Return:
        abs(f), px, py
    """
    vp1 = vpts[:, 0].reshape((-1, 1))
    vp2 = vpts[:, 1].reshape((-1, 1))
    vp3 = vpts[:, 2].reshape((-1, 1))

    f, px, py= symbols("f, px, py")
    K_inv = Matrix([[1/f, 0, -px/f], [0, 1/f, -py/f], [0, 0, 1]])

    # v_i^TK^-TK^-1v_j
    eq_0 = vp1.T@K_inv.T@K_inv@vp2
    eq_1 = vp1.T@K_inv.T@K_inv@vp3
    eq_2 = vp2.T@K_inv.T@K_inv@vp3

    f, px, py = solve([eq_0[0], eq_1[0], eq_2[0]], (f, px, py))[0]

    return abs(f), px, py

In [9]:
def get_rotation_mat(vpts: numpy.ndarray, f: float, u: float, v: float) -> numpy.ndarray:
    """
    Computes the rotation matrix using the camera parameters.
    """
    Z = vpts[:, 0].reshape((-1, 1))
    X = vpts[:, 1].reshape((-1, 1))
    Y = vpts[:, 2].reshape((-1, 1))
    
    K = numpy.array([[f, 0, u], [0, f, v], [0, 0, 1]]).astype(numpy.float64)
    K_inv = scipy.linalg.inv(a=K)

    # r_i = K^-1v_i
    r_1 = K_inv@X
    r_2 = K_inv@Y
    r_3 = K_inv@Z
    
    r_1 = r_1/scipy.linalg.norm(a=r_1)
    r_2 = r_2/scipy.linalg.norm(a=r_2)
    r_3 = r_3/scipy.linalg.norm(a=r_3)

    R = numpy.concatenate((r_1, r_2, r_3), axis=1)

    return R

In [10]:
def cross_ratio(ref_coords: numpy.ndarray, obj_coords: numpy.ndarray, line: numpy.ndarray) -> float:
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    # z-axis
    vp_z = vpts[:, 2]

    # ref coords
    ref_t = ref_coords[:, 0]
    ref_b = ref_coords[:, 1]

    # obj coords.
    r = obj_coords[:, 0]
    b = obj_coords[:, 1]

    line_ref_obj = numpy.cross(a=ref_b, b=b)
    v = numpy.cross(a=line_ref_obj, b=horizon_line)
    v = v/v[-1]
    
    line_v_ref_t = numpy.cross(a=v, b=ref_t)
    line_r_b = numpy.cross(a=r, b=b)
    t = numpy.cross(a=line_v_ref_t, b=line_r_b)
    t = t/t[-1]

    ratio = (scipy.linalg.norm(r-b)*scipy.linalg.norm(vp_z-t)/
             scipy.linalg.norm(t-b)/scipy.linalg.norm(vp_z-r))
    
    return ratio

# Main function

In [11]:
mat = numpy.asarray(a=PIL.Image.open('images/CSL.jpeg'))

# part 1
# get vanishing points for each of the directions
num_vpts = 3
vpts = numpy.zeros(shape=(3, num_vpts))
for i in range(num_vpts):
    print("[INFO]: Getting vanishing point %d" % i)
    # get at least three lines from user input
    n, lines, centers = get_input_lines(mat=mat)
    # solve for vanishing point
    vpts[:, i] = get_vanishing_pt(lines=lines)
    # plot the lines and the vanishing point
    plt_lines_vp(mat=mat, lines=lines, vp=vpts[:, i])

[INFO]: Getting vanishing point 0
[INFO]: Set at least 3 lines to compute vanishing point
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Getting vanishing point 1
[INFO]: Set at least 3 lines to compute vanishing point
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input


2023-11-27 10:06:19.950 python[1734:184422] +[CATransaction synchronize] called within transaction
2023-11-27 10:06:27.813 python[1734:184422] +[CATransaction synchronize] called within transaction
2023-11-27 10:06:53.579 python[1734:184422] +[CATransaction synchronize] called within transaction


[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Getting vanishing point 2
[INFO]: Set at least 3 lines to compute vanishing point
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input


2023-11-27 10:09:16.550 python[1734:184422] +[CATransaction synchronize] called within transaction


[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input
[INFO]: Click the two endpoints, use the right key to undo, and use the middle key to stop input


2023-11-27 10:10:15.802 python[1734:184422] +[CATransaction synchronize] called within transaction


In [12]:
# get the ground horizon line
horizon_line = get_horizon_line(vpts=vpts)
# plot the ground horizon line
plt_horizon_line(mat=mat, line=horizon_line)

2023-11-27 10:10:57.239 python[1734:184422] +[CATransaction synchronize] called within transaction


In [13]:
# part 2
# solve for the camera parameters (f, u, v)
f, u, v = get_camera_params(vpts=vpts)
print("[INFO]: Focal length   (f)   = %f" % f)
print("[INFO]: Optical center (u,v) = %f, %f" % (u, v))

[INFO]: Focal length   (f)   = 778.240162
[INFO]: Optical center (u,v) = 616.604950, 326.968426


In [14]:
# part 3
# solve for the rotation matrixåå
R = get_rotation_mat(vpts=vpts, f=f, u=u, v=v)
print("[INFO]: R Matrix")
print(R)

[INFO]: R Matrix
[[ 0.68661585 -0.0110659  -0.72693619]
 [-0.08934988  0.99101985 -0.09947993]
 [ 0.72150902  0.13325616  0.6794612 ]]


In [22]:
# part 4
# record image coordinates for each object and store in map
objs = ("Gable_0", "Gable_1")
coords = dict()
for obj in objs:
    coords[obj] = get_top_btm_coords(mat=mat, obj=obj)

print("[INFO]: Gable 0 coordinates")
print(coords["Gable_0"])
print("[INFO]: Gable 1 coordinates")
print(coords["Gable_1"])

# estimate heights
ratio = cross_ratio(ref_coords=coords["Gable_0"], obj_coords=coords["Gable_1"], line=horizon_line)
print("[INFO]: The cross ratio is %f" % ratio)

[INFO]: Click on the top coordinate of Gable_0
[INFO]: Click on the bottom coordinate of Gable_0
[INFO]: Click on the top coordinate of Gable_1
[INFO]: Click on the bottom coordinate of Gable_1
[INFO]: Gable 0 coordinates
[[508.03245865 507.06589659]
 [ 96.14044411 138.66917432]
 [  1.           1.        ]]
[INFO]: Gable 1 coordinates
[[900.9455353  899.03062179]
 [101.1373913  143.26548855]
 [  1.           1.        ]]
[INFO]: The cross ratio is 0.993542


2023-11-27 10:18:36.937 python[1734:184422] +[CATransaction synchronize] called within transaction
2023-11-27 10:18:41.381 python[1734:184422] +[CATransaction synchronize] called within transaction
2023-11-27 10:18:42.338 python[1734:184422] +[CATransaction synchronize] called within transaction
2023-11-27 10:18:42.600 python[1734:184422] +[CATransaction synchronize] called within transaction
2023-11-27 10:18:45.350 python[1734:184422] +[CATransaction synchronize] called within transaction
2023-11-27 10:18:48.655 python[1734:184422] +[CATransaction synchronize] called within transaction
2023-11-27 10:18:49.389 python[1734:184422] +[CATransaction synchronize] called within transaction
2023-11-27 10:18:49.656 python[1734:184422] +[CATransaction synchronize] called within transaction
